# Model Exploration

Testing the use of Style transfer models to simplify the image before processing it

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 07/01/2026   | Martin | Created   | Notebook to explore the use of Style Transfer models | 

# Content

* [Introduction](#introduction)

# Introduction

Use Image Style Transfer model to first transform images into a vector style before performing colour reduction and then contour finding

# AdaIN

- Model: Arbitrary Image Stylization (v1-256)
- Type: Encoder-Decoder with Adaptive Instance Normalisation
- Link: https://www.tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy
import cv2
import matplotlib.pyplot as plt

In [7]:
import transformers

In [ ]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  
  return img

In [16]:
img = load_img("./data/img1.jpg")

In [19]:
tf.shape(img)[:-1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([360, 640], dtype=int32)>

In [1]:
%load_ext watermark
%watermark

Last updated: 2026-01-07T16:05:53.032499+08:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.37.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.6.87.2-microsoft-standard-WSL2
Machine     : x86_64
Processor   : x86_64
CPU cores   : 20
Architecture: 64bit

